In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00


In [3]:
# all imports

import os
import re

from wordcloud import WordCloud, STOPWORDS

import numpy as np
import pandas as pd

import json
from datasets import load_dataset, DatasetDict, Dataset, load_metric
import random

In [4]:
def load_data(file):

  # Specify the path to your JSON file
  json_file_path = file
  # Initialize an empty dictionary
  data = {}

  # Load the JSON data into the dictionary
  try:
      with open(json_file_path, "r") as json_file:
          data = json.load(json_file)
      print("JSON file loaded successfully into a dictionary.")
  except FileNotFoundError:
      print(f"Error: JSON file '{json_file_path}' not found.")
  except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")
  except Exception as e:
      print(f"An error occurred: {e}")

  return data


In [ ]:
dpr_mt5_generated = Dataset.from_dict(dpr_mt5_emb)
print(dpr_mt5_generated)

print(dpr_mt5_generated[0]['question'])
print(dpr_mt5_generated[8]['question'])
print(dpr_mt5_generated[17]['question'])

Dataset({
    features: ['context', 'question'],
    num_rows: 5000
})
<extra_id_0> టి.వి.సీరియళ్లలోను నటించింది.
<extra_id_0> ప్రయోగ ప్రాజెక్టుగా ప్రారంభించారు.
<extra_id_0> 5 నుండి 10 కి.మీ. దూరంలో ఉంది.


In [5]:
file_path = r"/content/drive/MyDrive/IRE/train_chaii_hi_1.json"
train_file_path = r"/content/drive/MyDrive/IRE/train.json"
test_file_path = r"/content/drive/MyDrive/IRE/test.json"

In [ ]:
train_dict = load_data(train_file_path)
print("len(train_dict): ", len(train_dict))
print(train_dict[0].keys())
print(print(train_dict[0]['positive_ctxs']))

JSON file loaded successfully into a dictionary.
len(train_dict):  596
dict_keys(['question', 'answers', 'positive_ctxs', 'negative_ctxs', 'hard_negative_ctxs', 'q_id', 'lang'])
[{'title': '', 'text': 'चीनी साहित्य अपनी प्राचीनता, विविधता और ऐतिहासिक उललेखों के लिये प्रख्यात है। चीन का प्राचीन साहित्य "पाँच क्लासिकल" के रूप में उपलब्ध होता है जिसके प्राचीनतम भाग का ईसा के पूर्व लगभग 15वीं शताब्दी माना जाता है। इसमें इतिहास (शू चिंग), प्रशस्तिगीत (शिह छिंग), परिवर्तन (ई चिंग), विधि विधान (लि चि) तथा कनफ्यूशियस (552-479 ई.पू.) द्वारा संग्रहित वसंत और शरद-विवरण (छुन छिउ) नामक तत्कालीन इतिहास शामिल हैं जो छिन राजवंशों के पूर्व का एकमात्र ऐतिहासिक संग्रह है। पूर्वकाल में शासनव्यवस्था चलाने के लिये राज्य के पदाधिकारियों को कनफ्यूशिअस धर्म में पारंगत होना आवश्यक था, इससे सरकारी परीक्षाओं के लिये इन ग्रंथों का अध्ययन अनिवार्य कर दिया गया था।\nकनफ्यूशिअस के अतिरिक्त चीन में लाओत्स, चुआंगत्स और मेन्शियस आदि अनेक दार्शनिक हो गए हैं जिनके साहित्य ने चीनी जनजीवन को प्रभावित किया है।\n जनकवि चू य्वा

In [6]:
train_dict = load_data(train_file_path)
test_dict = load_data(test_file_path)
print("len(train_dict): ", len(train_dict))
print("len(test_dict): ", len(test_dict))

chai_dataset = {
  "train": {"question": [], "positivie_context": [], "negative_context": []},
  "test": {"question": [], "positivie_context": [], "negative_context": []},
}

def get_data(split_type, dict_type, chai_dataset, k):

  question_list = []
  positive_contest_list = []
  negative_contest_list = []
  random_seed = 42
  random.seed(random_seed)
  for i in dict_type:
    question_list.append(i['question'])
    positive_contest_list.append(i["positive_ctxs"][0]['text'])

  for i in range(len(positive_contest_list)):
    list1 =[]
    for j in range(len(positive_contest_list)):
      if(i != j):
        list1.append(positive_contest_list[j])
    list1 = random.choices(list1, k=k)
    negative_contest_list.append(list1[:])

  for i in range(len(question_list)):
    chai_dataset[split_type]["question"] = question_list
    chai_dataset[split_type]["positivie_context"] = positive_contest_list
    chai_dataset[split_type]["negative_context"] = negative_contest_list

#build train
get_data("train", train_dict, chai_dataset, 10)

#build test
get_data("test", test_dict, chai_dataset, 10)


JSON file loaded successfully into a dictionary.
JSON file loaded successfully into a dictionary.
len(train_dict):  596
len(test_dict):  149


In [7]:
print(type(chai_dataset))
print(type(chai_dataset['train']))

dpr_dataset = Dataset.from_dict(chai_dataset['train'])
print(dpr_dataset)
print(dpr_dataset[0])
print(dpr_dataset[100])
print(dpr_dataset[200])
print(dpr_dataset[300])
print(dpr_dataset[400])

<class 'dict'>
<class 'dict'>
Dataset({
    features: ['question', 'positivie_context', 'negative_context'],
    num_rows: 596
})
{'question': 'चीन के सर्वप्रथम जनकवि किसे माना जाता हैं?', 'positivie_context': 'चीनी साहित्य अपनी प्राचीनता, विविधता और ऐतिहासिक उललेखों के लिये प्रख्यात है। चीन का प्राचीन साहित्य "पाँच क्लासिकल" के रूप में उपलब्ध होता है जिसके प्राचीनतम भाग का ईसा के पूर्व लगभग 15वीं शताब्दी माना जाता है। इसमें इतिहास (शू चिंग), प्रशस्तिगीत (शिह छिंग), परिवर्तन (ई चिंग), विधि विधान (लि चि) तथा कनफ्यूशियस (552-479 ई.पू.) द्वारा संग्रहित वसंत और शरद-विवरण (छुन छिउ) नामक तत्कालीन इतिहास शामिल हैं जो छिन राजवंशों के पूर्व का एकमात्र ऐतिहासिक संग्रह है। पूर्वकाल में शासनव्यवस्था चलाने के लिये राज्य के पदाधिकारियों को कनफ्यूशिअस धर्म में पारंगत होना आवश्यक था, इससे सरकारी परीक्षाओं के लिये इन ग्रंथों का अध्ययन अनिवार्य कर दिया गया था।\nकनफ्यूशिअस के अतिरिक्त चीन में लाओत्स, चुआंगत्स और मेन्शियस आदि अनेक दार्शनिक हो गए हैं जिनके साहित्य ने चीनी जनजीवन को प्रभावित किया है।\n जनकव

In [ ]:
# files = os.listdir('data')
# print(len(files))

# # keeping only first 15 txt files
# first = 80
# documents = list(filter(lambda file: file[-4:] == '.txt', files))
# keys = list(filter(lambda file: file[-4:] == '.key', files))
# # files = files[:first]
# print(len(files))

# def getDataFromFile(file):
#   with open(file, 'r', encoding='utf-8', errors='ignore') as f:
#     lines = f.readlines()

#   lines = list(filter(lambda line: line not in ['\n', ' \n'] , lines))
#   # return lines
#   fileData = ""
#   for line in lines:
#     fileData += " " + line
#   return fileData

# data = list(map(lambda document: getDataFromFile('data/' + document), documents))
# print(documents)
# # print(keys)

# def getStopWords(filePath):
#   with open(filePath) as f:
#     lines = f.readlines()
#   text = ""
#   for line in lines:
#     line.strip()
#     text = text + ' ' + line[:-1]
#   return text.split()
# stopWords = getStopWords('english.stop')

282
282
['mip14195.txt', 'str12695.txt', 'eos07195.txt', 'eos20195.txt', 'sbr12195.txt', 'mip09195.txt', 'emt05095.txt', 'emt04895.txt', 'str19495.txt', 'mip18495.txt', 'inf17195.txt', 'sbr01495.txt', 'ins03495.txt', 'sbr15695.txt', 'eos16095.txt', 'str00295.txt', 'emt04795.txt', 'sbr17895.txt', 'ins15795.txt', 'sbr19195.txt', 'ins20795.txt', 'inf12495.txt', 'eos19595.txt', 'emt01995.txt', 'emt10195.txt', 'sbr16395.txt', 'mip14795.txt', 'eos11695.txt', 'eos00395.txt', 'str02595.txt', 'inf12995.txt', 'eos07795.txt', 'mip02195.txt', 'str07995.txt', 'str10895.txt', 'ins19295.txt', 'inf07395.txt', 'inf144b95.txt', 'mat06395.txt', 'sbr17995.txt', 'ins07595.txt', 'emt07295.txt', 'mat02395.txt', 'sbr21495.txt', 'mip00195.txt', 'eos19895.txt', 'ins20495.txt', 'emt07895.txt', 'str01895.txt', 'emt13895.txt', 'emt17495.txt', 'inf12795.txt', 'inf11495.txt', 'mip15495.txt', 'ins05895.txt', 'eos03695.txt', 'emt10395.txt', 'inf11595.txt', 'mat00695.txt', 'eos03995.txt', 'emt02495.txt', 'sbr06195.txt'

In [16]:
import regex as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Example Hindi text
text = "यह एक उदाहरण पाठ है।"

# Cleaning
text = re.sub(r'[^\p{Devanagari} ]+', '', text)
text = re.sub(r'\p{Mn}', '', text)

# Tokenization
nltk.download('punkt')
tokens = word_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
hindi_stopwords = [
    'और', 'का', 'कि', 'है', 'में', 'की', 'को', 'पर', 'हैं',
    'से', 'ने', 'कर', 'तो', 'ही', 'या', 'था', 'द्वारा', 'होता',
    'किया', 'गया', 'किये', 'लिए', 'साथ', 'करने', 'किसी', 'इस',
    'होती', 'इसका', 'इसे', 'इसके', 'सभी', 'इन', 'इनके', 'इनका', 'इनकी',
    'इत्यादि'
]

# Pre Processing
def helper_preprocess(text):
    # Remove Punctuation and Numbers:
    text = re.sub(r'[^\p{Devanagari} ]+', '', text)

    # Normalize Unicode Characters:
    text = re.sub(r'\p{Mn}', '', text)

    # Whitespace Normalization: Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenization: Split the text into words (tokens)
    # Basic tokenization based on whitespace. For more advanced tokenization, use a Telugu-specific tokenizer
    tokens = text.split()
    tokens = list(filter(lambda token : token not in hindi_stopwords, tokens))

    # return ' '.join(tokens)
    return tokens

# print(helper_preprocess(text))

# def preprocess_function(examples):
#     inputs = [helper_preprocess(doc) for doc in examples["context"]]
#     examples["context"] = inputs

#     inputs = [helper_preprocess(doc) for doc in examples["question"]]
#     examples["question"] = inputs
#     return examples

documents = []
docIndex = []
for i in range(500):
  try:
    documents.append(helper_preprocess(dpr_dataset[i]['positivie_context']))
    docIndex.append(i)
  except:
    continue

In [15]:
print(len(documents))
print(len(docIndex))
# print(documents)

500
500


In [21]:
import tqdm

class TFIDF:
  def __init__(self, documents, data):
    self.documents = documents[:100]
    self.data = data[:100]
    self.dicti = list()
    self.totalDocs = len(self.documents)
    self.termsInDicti = 0
    self.tfidf = list()
    self.idf = list()
    self.tf = list()
    self.createWordsDict()
    self.inverseDocumentFreq()
    self.tfidfMatrix()

  def createWordsDict(self):
    dicti = []
    cnt = 0
    print("creating dictionary for the data")
    for document in self.data:
      cnt += len(document)
      for token in document:
        if token not in dicti:
          dicti.append(token)

    print("total words:", cnt)
    self.dicti = dicti
    self.termsInDicti = len(dicti)
    print("total unique words: ", self.termsInDicti)

  def inverseDocumentFreq(self):
    idfList = {}

    for term in self.dicti:
      df = 0
      for document in self.data:
        if term in document:
          df += 1
      idf = np.log(self.totalDocs / df)
      idfList[term] = idf
    self.idf = idfList

  def tfidfMatrix(self):
    tfidfMatrix = []
    # tfidfMatrix['files'] = self.documents

    print("creating tfidf matrix for the document")
    for document in self.data:
      tfidfDocument = list()

      for term in self.dicti:
        tf = document.count(term)
        tfidfDocument.append(np.log(1 + tf) * self.idf[term])

      tfidfMatrix.append(tfidfDocument)

    self.tfidf = tfidfMatrix
    # print(len(self.tfidf))
    # print(len(self.tfidf[0]))

  def queryRepresentation(self, queryText):
    preprocess_query = helper_preprocess(queryText)

    tfidfQuery = []
    for term in self.dicti:

      tf = preprocess_query.count(term)
      tfidfQuery.append(np.log(1 + tf) * self.idf[term])

    return tfidfQuery


  def similarityMeasure(self, query, k):

    querytfidfRep = self.queryRepresentation(query)
    simiScores = []
    index = 0

    for i in range(self.totalDocs):
      doctfidfRep = self.tfidf[i]

      intersection = np.logical_and(doctfidfRep, querytfidfRep)
      union = np.logical_or(doctfidfRep, querytfidfRep)
      # Jaccard similarity
      simiScores.append(intersection.sum() / float(union.sum()))

    documentsRankingIndices = np.argsort(simiScores)[::-1]
    documentRanking = list(list(map(lambda i : self.documents[i], documentsRankingIndices)))

    if len(documentRanking) > k:
      return documentRanking[:k], simiScores[:k]
    return documentRanking, simiScores

print("After basic preprocessing ")
tfidfModel1 = TFIDF(docIndex, documents)

# df1 = tfidfModel1.getTFIDF()
# topPStemsWithStopwords = tfidfModel1.topPStemsCorpus(7000)
# print("top 2000 stems in the corpus: ", topPStemsWithStopwords)

After basic preprocessing 
creating dictionary for the data
total words: 163251
total unique words:  20189
creating tfidf matrix for the document


In [66]:

def generate_testcases(n, k):
    random_numbers = [random.randint(0, n) for _ in range(k)]
    random_numbers

    queries = []
    for r in random_numbers:
      queries.append(dpr_dataset[r]['question'])

    return queries, random_numbers

queries, actual_doc = generate_testcases(100, 5)
predictions = []

In [67]:
print(dpr_dataset[actual_doc[0]])
rankings, similarityScores = tfidfModel1.similarityMeasure(queries[0], 5)

print("actual_doc: ", actual_doc[0])
predictions.append(rankings)
for i in range(len(rankings)):
  print(f'Rank {i} -> {rankings[i]}')

{'question': 'कुतुब मीनार की ऊंचाई कितनी है?', 'positivie_context': 'कुतुब समूह के अन्य उल्लेखनीय स्थलों एवं निर्माणों हेतु देखें मुख्य लेख\nकुतुब मीनार भारत में दक्षिण दिल्ली शहर के महरौली भाग में स्थित, ईंट से बनी विश्व की सबसे ऊँची मीनार है। इसकी ऊँचाई और व्यास १४.३ मीटर है, जो ऊपर जाकर शिखर पर हो जाता है। इसमें ३७९ सीढियाँ हैं।[1] मीनार के चारों ओर बने अहाते में भारतीय कला के कई उत्कृष्ट नमूने हैं, जिनमें से अनेक इसके निर्माण काल सन 1192 के हैं। यह परिसर युनेस्को द्वारा विश्व धरोहर के रूप में स्वीकृत किया गया है।\nइतिहास अफ़गानिस्तान में स्थित, जाम की मीनार से प्रेरित एवं उससे आगे निकलने की इच्छा से, दिल्ली के प्रथम मुस्लिम शासक कुतुबुद्दीन ऐबक, ने इस्लाम फैलाने की सनक के कारण वेदशाला को तोड़कर कुतुब मीनार का पुनर्निर्माण सन ११९३ में आरम्भ करवाया, परन्तु केवल इसका आधार ही बनवा पाया। उसके उत्तराधिकारी इल्तुतमिश ने इसमें तीन मंजिलों को बढ़ाया और सन १३६८ में फीरोजशाह तुगलक ने पाँचवीं और अन्तिम मंजिल बनवाई। । मीनार को लाल बलुआ पत्थर से बनाया गया है, जिस पर कुरान की आयतों की एवं फूल बेल

In [68]:
print(dpr_dataset[actual_doc[1]])
rankings, similarityScores = tfidfModel1.similarityMeasure(queries[1], 5)

print("actual_doc: ", actual_doc[1])
predictions.append(rankings)
for i in range(len(rankings)):
  print(f'Rank {i} -> {rankings[i]}')

{'question': 'एशियाई क्रिकेट परिषद का मुख्यालय कहाँ पर है?', 'positivie_context': 'एशियाई क्रिकेट परिषद (एसीसी) एक क्रिकेट संगठन है जो 1983 में स्थापित किया गया था, को बढ़ावा देने और एशिया में क्रिकेट के खेल को विकसित करने के लिए है। अंतरराष्ट्रीय क्रिकेट परिषद के अधीनस्थ, परिषद महाद्वीप के क्षेत्रीय प्रशासनिक निकाय है, और वर्तमान में 25 सदस्य संघों के होते हैं। शहरयार खान एशियाई क्रिकेट परिषद के वर्तमान अध्यक्ष हैं।[1]\n\nइतिहास\nएसीसी पहले कुआलालंपुर, मलेशिया में मुख्यालय था, परिषद मूल रूप से एशियाई क्रिकेट सम्मेलन के रूप में 1983 में स्थापित किया गया था 1995 में वर्तमान के लिए इसका नाम बदल रहा है। 2003 तक, परिषद के मुख्यालय द्विवार्षिक \'राष्ट्रपति और सचिवों\' घर देशों के बीच घुमाया गया। संगठन के वर्तमान अध्यक्ष मुस्तफा कमाल, जो बांग्लादेश क्रिकेट बोर्ड के पूर्व अध्यक्ष है और अब वह अंतरराष्ट्रीय क्रिकेट परिषद के उपाध्यक्ष है।\nपरिषद के एक विकास कार्यक्रम है कि सदस्य देशों में कोचिंग, अंपायरिंग और खेल चिकित्सा कार्यक्रमों का समर्थन करता है, आधिकारिक तौर पर स्वीकृत एशिया कप, एशियाई टे

In [69]:
print(dpr_dataset[actual_doc[2]])
rankings, similarityScores = tfidfModel1.similarityMeasure(queries[2], 5)

print("actual_doc: ", actual_doc[2])
predictions.append(rankings)
for i in range(len(rankings)):
  print(f'Rank {i} -> {rankings[i]}')

{'question': 'इस्लामी सैन्य काउंटर टेररिज्म कोएलिशन की स्थापना किसने की थी?', 'positivie_context': 'इस्लामी सैन्य गठबंधन: (आईएमए) (अरबी: التحالف الإسلامي العسكري لمحاربة الإرهاب), और इस्लामी सैन्य काउंटर टेररिज्म कोएलिशन (आईएमसीटीसी) के रूप में भी जाना जाता है, मुस्लिम दुनिया  में एक अंतर-सरकारी आतंकवाद विरोधी गठबंधन है, जो सेना के चारों ओर एकजुट है आईएसआईएस और अन्य आतंकवादी गतिविधियों के खिलाफ हस्तक्षेप।.[1][2] इसकी रचना पहले सऊदी अरब के रक्षा मंत्री मोहम्मद बिन सलमान अल सऊद ने 15 दिसंबर 2015 को घोषित की थी।[3] गठबंधन के लिए रियाद, सऊदी अरब में एक संयुक्त अभियान केंद्र था। जब गठबंधन की घोषणा की गई तो 34 सदस्य थे। और अतिरिक्त देश  भी शामिल हुए और सदस्यों की संख्या 41 हो गई जब ओमान दिसंबर 2016 में शामिल हुआ। 6 जनवरी 2017 को, पाकिस्तानी सेना प्रमुख जनरल (सेवानिवृत्त) रहेल शरीफ को आईएमसीटीसी का पहले कमांडर-इन-चीफ का नाम दिया गया था।.[4][5]\nइतिहास और उद्देश्य\nआईएमसीटीसी ने कहा है कि इसका प्राथमिक उद्देश्य सभी संप्रदाय की आतंकवादी समूहों और आतंकवादी संगठनों से मुस्लिम देशों की रक्षा करना 

In [70]:
print(dpr_dataset[actual_doc[3]])
rankings, similarityScores = tfidfModel1.similarityMeasure(queries[3], 5)

print("actual_doc: ", actual_doc[3])
predictions.append(rankings)
for i in range(len(rankings)):
  print(f'Rank {i} -> {rankings[i]}')

{'question': 'आई एन एस विक्रमादित्य युद्धपोत किस कंपनी द्वारा निर्मित किया गया है?', 'positivie_context': 'Main Page\n\nआई एन एस (INS) विक्रमादित्य  (Sanskrit: विक्रमादित्य, Vikramāditya, "सूर्य की तरह प्रतापी") पूर्व सोवियत विमान वाहक एडमिरल गोर्शकोव का नया नाम है, जो भारत द्वारा हासिल किया गया है। पहले अनुमान था कि 2012 में इसे भारत को सौंप दिया जाएगा, किंतु काफी विलंब[5] के पश्चात् 16 नवम्बर 2013 को इसे भारतीय नौसेना में सेवा के लिए शामिल कर लिया गया।[1][6] दिसंबर अंत या जनवरी आरंभ में यह भारतीय नौसैनिक अड्डा कारवाड़ तक पहुंच जाएगा।[3]\nविक्रमादित्य यूक्रेन के माइकोलैव ब्लैक \u200dसी शिपयार्ड में 1978-1982 में निर्मित कीव श्रेणी के विमान वाहक पोत का एक रूपांतरण है। रूस के अर्खान्गेल्स्क ओब्लास्ट के सेवेरॉद्विनस्क के सेवमाश शिपयार्ड में इस जहाज की बड़े स्तर पर मरम्मत की गई। यह पोत भारत के एकमात्र सेवारत विमान वाहक पोत आई एन एस विराट का स्थान लेगा। इस पोत को नया रूप देने में भारत को 2.3 अरब डॉलर खर्च करने पड़े हैं।[3]\n क्षमता \nविक्रमादित्य ४५३०० टन भार वाला, २८४ मीटर लम्बा और ६० मीट

In [71]:
print(dpr_dataset[actual_doc[4]])
rankings, similarityScores = tfidfModel1.similarityMeasure(queries[4], 5)

print("actual_doc: ", actual_doc[4])
predictions.append(rankings)
for i in range(len(rankings)):
  print(f'Rank {i} -> {rankings[i]}')

{'question': 'अंटार्कटिका नाम का पहले पहल औपचारिक प्रयोग किसने किया था?', 'positivie_context': 'अंटार्कटिका\nअंतरिक्ष से अंटार्कटिका का दृश्य।\nलाम्बर्ट अजिमुथाल प्रोजेक्शन पर आधारित मानचित्र।\nदक्षिणी ध्रुव मध्य में है।\nक्षेत्र (कुल) (बर्फ रहित)\n(बर्फ सहित)\n14,000,000 किमी2 (5,000,000 वर्ग मील) 280,000 किमी2 (100,000 वर्ग मील) 13,720,000 किमी2 (5,300,000 वर्ग मील) जनसंख्या (स्थाई) (अस्थाई) 7वां\nशून्य ≈1,000 निर्भर 4\n· Bouvet Island · French Southern Territories · Heard Island and McDonald Islands · South Georgia and the South Sandwich Islands\nआधिकारिक क्षेत्रीय दावें अंटार्कटिक संधि व्यवस्था 8\n· Adelie Land · Antártica · Argentine Antarctica · Australian Antarctic Territory · British Antarctic Territory · Queen Maud Land · Peter I Island · Ross Dependency\nअनाधिकृत क्षेत्रीय दावें 1\nBrazilian Antarctica\nदावों पर अधिकार के लिए सुरक्षित 2\n· Russian Federation · United States of America\nटाइम जोन नहीं\nUTC-3 (केवल ग्राहम लैंड) इंटरनेट टाप लेवल डामिन .aq कालिंग कोड प्रत्येक बेस 

In [73]:
def GetTestPerformance( Org_Questions, retrieved_DocId, y_true):

    Num_Correct = 0;
    Reciprocal_Rank = 0
    idx = 0
    for i in range(len(Org_Questions)):
        q = Org_Questions[i]
        # p_ctx = p_ctx[0]
        for j in range(len(retrieved_DocId)):
          # print()
            if retrieved_DocId[i][j] == y_true[i]:
                Num_Correct += 1
                Reciprocal_Rank += 1/(j+1)
                break

        idx += 1

    Accuracy = Num_Correct/(idx+1)
    MRR = Reciprocal_Rank/(idx+1)
    return Accuracy, MRR

print("queries: ", queries)
print("top 5 predicted docs:", predictions)
print("actual_doc: ", actual_doc)
acc, mrr = GetTestPerformance(queries, predictions, actual_doc)

print()
print(f'TFIDF Train Accruacy -> {acc}')
print(f'TFIDF Train MRR -> {mrr}')

queries:  ['कुतुब मीनार की ऊंचाई कितनी है?', 'एशियाई क्रिकेट परिषद का मुख्यालय कहाँ पर है?', 'इस्लामी सैन्य काउंटर टेररिज्म कोएलिशन की स्थापना किसने की थी?', 'आई एन एस विक्रमादित्य युद्धपोत किस कंपनी द्वारा निर्मित किया गया है?', 'अंटार्कटिका नाम का पहले पहल औपचारिक प्रयोग किसने किया था?']
top 5 predicted docs: [[75, 11, 63, 5, 56], [51, 75, 24, 11, 5], [77, 56, 32, 87, 38], [44, 51, 33, 75, 55], [51, 75, 11, 63, 93]]
actual_doc:  [3, 18, 77, 86, 100]

TFIDF Train Accruacy -> 0.16666666666666666
TFIDF Train MRR -> 0.16666666666666666
